# Chapter 6. Object References, Mutability, and Recycling

Distinction between objects and their names. A name is not the object; a name is a separate thing.

Variables are labels, not boxes.

Tuple: they are immutable but their values may change. --> Discussion of shallow and deep copies. References and function parameters are our next theme: the problem with mutable parameter defaults and the safe handling of mutable arguments passed by clients of our functions.

The last sections of the chapter cover garbage collection, the `del` command, and a selection of tricks that Python plays with immutable objects.

## Variables Are Not Boxes

Python variables = reference variables in Java

--> Thay vì nghĩ các biến là box, coi nó là 1 labels thì tốt hơn

Hình 6-1 minh họa lý do tại sao phép ẩn dụ hộp không phù hợp với Python

![img.png](image/6-1.png)

sẽ hợp lý hơn nhiều khi nói rằng biến đó được gán cho một đối tượng chứ không phải ngược lại.

--> Sẽ hợp lý hơn nhiều khi nói rằng biến đó được gán cho một đối tượng chứ không phải ngược lại.

In [1]:
a = [1, 2, 3]
b = a
a.append(4)
b

[1, 2, 3, 4]

After all, the object is created before the assignment

In [2]:
# Example 6-2. Variables are bound to objects only after the objects are created

class Gizmo:
    def __init__(self):
        print(f'Gizmo id: {id(self)}')

x = Gizmo()

Gizmo id: 140545971434640


In [3]:
y = Gizmo() * 10 # Nhân Gizmo instance will raise an exception

# Gizmo id dưới được tin ra chứng minh nó được tạo trước khi nhân

Gizmo id: 140545971436128


TypeError: unsupported operand type(s) for *: 'Gizmo' and 'int'

In [4]:
dir() # biến y không được tạo, vì xảy ra Exception ở bên phải

['Gizmo',
 'In',
 'Out',
 '_',
 '_1',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_dh',
 '_i',
 '_i1',
 '_i2',
 '_i3',
 '_i4',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 '_pydevd_bundle',
 'a',
 'b',
 'exit',
 'get_ipython',
 'open',
 'pydev_jupyter_vars',
 'quit',
 'remove_imported_pydev_package',
 'sys',
 'x']

Bởi vì các biến chỉ là các label, không có gì ngăn cản một object được gán nhiều label. Khi điều đó xảy ra, ta có `aliasing`

##  Identity - id(), Equality - ==, and Aliases - =

Example 6-3. charles and lewis refer to the same object.

Example 6-3 is an example of aliasing. In that code, lewis and charles are aliases: two variables bound to the same object.

In [5]:
charles = {'name': 'Charles L. Dodgson', 'born': 1832}
lewis = charles
lewis is charles

True

In [6]:
id(charles), id(lewis) # returns the memory address of the object

(140545974635904, 140545974635904)

In [7]:
lewis['balance'] = 950
charles

{'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}

![img.png](image/6-2.png)

In [8]:
alex = {'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}

alex == charles # __eq__ implementation in the dict class.

True

In [9]:
alex is charles

False

*Note:*

- An object’s identity never changes once it has been created; you may think of it as the object’s address in memory. The is operator compares the identity of two objects; the `id()` function returns an integer representing its identity.

- The `==` operator compares the values of objects (the data they hold), while `is()` compares their identities.


### The Relative Immutability of Tuples

`Tuples - ()`, like most Python collections—`lists []`, `dicts - {k:v}`, `sets - {a,b,c}`, etc.—are containers: they hold references to objects

In [10]:
t1 = (1, 2, [30, 40])
t2 = (1, 2, [30, 40])

t1 == t2

True

In [11]:
t1[-1]

[30, 40]

In [12]:
id(t1[-1])

140545974644992

In [13]:
t1[-1].append(99)

In [14]:
t1

(1, 2, [30, 40, 99])

In [15]:
t2

(1, 2, [30, 40])

## Copies Are Shallow by Default

The easiest way to copy is to use the built-in constructor for the type itself.

l2 = list(l1)
l3 = l1[:]

 `shallow copy` (i.e., the outermost container is duplicated, but the copy is filled with references to the same items held by the original container).

Chỉ là tạo ra 1 ref mới, chứa các ref cũ ở trong

In [16]:
l1 = [3, [55, 44], (7, 8, 9)]
l2 = list(l1)
l2

[3, [55, 44], (7, 8, 9)]

In [17]:
l2 is l1

False

In [18]:
l3 = l1[:] # shallow copy
l3

[3, [55, 44], (7, 8, 9)]

Example 6-6. Making a shallow copy of a list containing another list

In [20]:
l1 = [3, [66, 55, 44], (7, 8, 9)]
l2 = list(l1)
l3 = l1[:]
l1.append(100)
l1[1].remove(55)

print('l1:', l1)
print('l2:', l2)
print('l3:', l3)

l1: [3, [66, 44], (7, 8, 9), 100]
l2: [3, [66, 44], (7, 8, 9)]
l3: [3, [66, 44], (7, 8, 9)]


l2, l3 là các ref mới, chứa ref value của l1
--> l1 đc append . l2,l3 kệ
--> ref value l1[1] được xoá. l2,l3 ăn theo

In [21]:
l2[1] += [33, 22]
l2[2] += (10, 11)

print('l1:', l1)
print('l2:', l2)
print('l3:', l3)

l1: [3, [66, 44, 33, 22], (7, 8, 9), 100]
l2: [3, [66, 44, 33, 22], (7, 8, 9, 10, 11)]
l3: [3, [66, 44, 33, 22], (7, 8, 9)]


l2 thay đổi:
- l2[1] là list (mutable) --> ko tạo ra ref value mới --> l1, l3 ăn theo ref đó
- l2[2] là tuple (immutable) --> tạo ra 1 ref mới --> l1, l3 ko ăn theo ref đó

==> Shallow Copy khá nguy hiểm khi tạo nên những thay đổi ko đáng có ở 1 biến.

### Deep and Shallow Copies of Arbitrary Objects

deep copies (i.e., duplicates that do not share references of embedded objects).

The copy module provides the `deepcopy` and `copy` functions that return deep and shallow copies of arbitrary objects.

Example 6-8. Bus picks up and drops off passengers

In [22]:
class Bus:

    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)

In [28]:
import copy
bus1 = Bus(['Alice', 'Bill', 'Claire', 'David'])
bus2 = copy.copy(bus1)
bus3 = copy.deepcopy(bus1)

id(bus1), id(bus2), id(bus3)

(140545974386496, 140546378191856, 140546378197664)

In [31]:
id(bus1.passengers), id(bus2.passengers), id(bus3.passengers) # id(bus3.passengers) khác

(140545970234624, 140545970234624, 140546378626240)

In [29]:
bus1.drop('Bill')
bus2.passengers

['Alice', 'Claire', 'David']

In [30]:
bus3.passengers

['Alice', 'Bill', 'Claire', 'David']

The `deepcopy` function remembers the objects already copied to handle cyclic references.

In [32]:
a = [10, 20]
b = [a, 30]
a.append(b)
a

[10, 20, [[...], 30]]

In [37]:
a[2]

[[10, 20, [...]], 30]

In [41]:
from copy import deepcopy
c = deepcopy(a)
c

[10, 20, [[...], 30]]

Also, a deep copy may be too deep in some cases.

For example, objects may refer to external resources or singletons that should not be copied. You can control the behavior of both copy and deepcopy by implementing the `__copy__()` and `__deepcopy__()` special methods.
(https://docs.python.org/3/library/copy.html)

## Function Parameters as References

The only mode of parameter passing in Python is `call by sharing` (the parameters inside the function become aliases (id(a) bằng id(b) và cùng ref đến 1 giá trị) of the actual arguments).

-->  a function may change any mutable object passed as a parameter, but it cannot change the identity of those objects.

In [42]:
def f(a, b):
    a += b
    return a

x = 1
y = 2
f(x,y)

3

In [43]:
# number x is unchanged.
x, y

(1, 2)

In [44]:
a = [1, 2]
b = [3, 4]
f(a, b)

[1, 2, 3, 4]

In [48]:
# list (mutable) a is changed.
a, b

([1, 2, 3, 4], [3, 4])

In [46]:
t = (10, 20)
u = (30, 40)
f(t, u)

(10, 20, 30, 40)

In [47]:
# tuple t is unchanged.
t, u

((10, 20), (30, 40))

### Mutable Types as Parameter Defaults: Bad Idea

Should avoid mutable objects as default values for parameters.

In [49]:
# Example 6-12. A simple class to illustrate the danger of a mutable default

class HauntedBus:
    """A bus model haunted by ghost passengers"""

    def __init__(self, passengers=[]):  # self.passengers sẽ thành alias + default value khi passengers đc khởi tạo rỗng. Xem cách khởi tạo đúng ở class Bus
        self.passengers = passengers

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)

In [50]:
bus1 = HauntedBus(['Alice', 'Bill'])
bus1.passengers

['Alice', 'Bill']

In [51]:
bus1.pick('Charlie')
bus1.drop('Alice')
bus1.passengers

['Bill', 'Charlie']

In [52]:
bus2 = HauntedBus()
bus2.pick('Carrie')
bus2.passengers

['Carrie']

In [53]:
bus3 = HauntedBus()
bus3.passengers # The default is no longer empty!

['Carrie']

In [54]:
bus3.pick('Dave')
bus2.passengers

['Carrie', 'Dave']

The problem: `bus2.passengers` and `bus3.passengers` refer to the same list. But `bus1.passengers` is a distinct list.

--> The problem is that `HauntedBus` instances that don’t get an initial passenger list end up sharing the same passenger list among themselves.

Khi HauntedBus được khởi tạo với hành khách, nó sẽ hoạt động như mong đợi. Những điều kỳ lạ chỉ xảy ra khi HauntedBus bắt đầu với empty list, khi đó `self.passengers` trở thành 1 alias cho default `passengers`

In [55]:
dir(HauntedBus.__init__)  # doctest: +ELLIPSIS

['__annotations__',
 '__builtins__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [56]:
HauntedBus.__init__.__defaults__ # default ko còn là [] nữa

(['Carrie', 'Dave'],)

In [59]:
# verify that bus2.passengers is an alias bound to the first element of the HauntedBus.__init__.__defaults__ attribute:
HauntedBus.__init__.__defaults__[0]

['Carrie', 'Dave']

In [60]:
HauntedBus.__init__.__defaults__[0] is bus2.passengers

True

==> explains why `None` is commonly used as the default value for parameters that may receive mutable values.

```
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)
```

### Defensive Programming with Mutable Parameters

Khi coding một hàm nhận một mutable parameter, nên xem xét cẩn thận liệu người gọi có mong đợi đối số được truyền vào bị thay đổi hay không.

In [61]:
# Example 6-14: thành viên đội bóng rổ bị xoá, khi họ xuống xe Bus :D

class TwilightBus:
    """A bus model that makes passengers vanish"""

    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = passengers # makes self.passengers an alias for passengers --> self.passengers và basketball_team trỏ tới cùng giá trị

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)

basketball_team = ['Sue', 'Tina', 'Maya', 'Diana', 'Pat']
bus = TwilightBus(basketball_team)
bus.drop('Tina')

basketball_team # Mất Tina

['Sue', 'Maya', 'Diana', 'Pat']

--> Tip nếu không muốn `basketball_team` bị thay đổi

```
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers) # Deep copy: Make a copy of the passengers list, or convert it to a list if it’s not one
```

## del and Garbage Collection

"Objects are never explicitly (rõ ràng) destroyed; however, when they become unreachable they may be garbage-collected."

- The first fact about `del` is that it’s not a function, it’s a statement. `del x` is not `del(x)`
- The second fact is that `del` deletes references, not objects. (khi ko còn tham chiếu đến object -> nó sẽ đc thu gom và xoá)


## Tricks Python Plays with Immutables

In [62]:
# Example 6-17. A tuple built from another is actually the same exact tuple
t1 = (1, 2, 3)
t2 = tuple(t1) # shallow copy
t3 = t1[:] # shallow copy

id(t1), id(t2), id(t3)

(140545974827520, 140545974827520, 140545974827520)

In [64]:
t2 is t1 , t3 is t1

(True, True)

The same behavior can be observed with instances of `str`, `bytes`, and `frozenset`.

Note that a `frozenset` is not a sequence, so `fs[:]` does not work if fs is a `frozenset`. But `fs.copy()` has the same effect

In [66]:
t1 = (1, 2, 3)
t3 = (1, 2, 3)
t3 is t1 # t1 and t3 are equal, but not the same object.

False

In [67]:
s1 = 'ABC'
s2 = 'ABC'
s2 is s1 # a and b refer to the same str!

True

The sharing of string literals is an optimization technique called `interning`
CPython uses a similar technique with small integers to avoid unnecessary duplication

WARNING
Never depend on str or int interning! Always use `==` instead of `is` to compare strings or integers for equality. Interning is an optimization for internal use of the Python interpreter.

In [68]:
s1 == s2

True

--> “lời nói dối” vô hại ở phần này (`interning` và `frozenset.copy()`) giúp tiết kiệm bộ nhớ và giúp trình thông dịch nhanh hơn.

# Chapter Summary

Every Python object has an identity, a type, and a value. Only the value of an object may change over time
